# Hands-on session 1 - Variational Auto-Encoders
## Generative Modeling Summer School 2024

### Theory behind VAEs

VAEs are latent variable models trained with variational inference. In general, the latent variable models define the following generative process:
\begin{align}
1.\ & \mathbf{z} \sim p_{\lambda}(\mathbf{z}) \\
2.\ & \mathbf{x} \sim p_{\theta}(\mathbf{x}|\mathbf{z})
\end{align}

In plain words, we assume that for observable data $\mathbf{x}$, there are some latent (hidden) factors $\mathbf{z}$. Then, the training objective is the log-likelihood function of the following form:
$$
\log p_{\vartheta}(\mathbf{x})=\log \int p_\theta(\mathbf{x} \mid \mathbf{z}) p_\lambda(\mathbf{z}) \mathrm{d} \mathbf{z} .
$$

The problem here is the intractability of the integral if the dependencies between random variables $\mathbf{x}$ and $\mathbf{z}$ are non-linear and/or the distributions are non-Gaussian.

By introducing variational posteriors $q_{\phi}(\mathbf{z}|\mathbf{x})$, we get the following lower bound (the Evidence Lower Bound, ELBO):
$$
\log p_{\vartheta}(\mathbf{x}) \geq \mathbb{E}_{\mathbf{z} \sim q_\phi(\mathbf{z} \mid \mathbf{x})}\left[\log p_\theta(\mathbf{x} \mid \mathbf{z})\right]-\mathrm{KL}\left(q_\phi(\mathbf{z} \mid \mathbf{x}) \| p_\lambda(\mathbf{z})\right) .
$$

## IMPORTS

In [23]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install pytorch_model_summary

Note: you may need to restart the kernel to use updated packages.


In [25]:
# DO NOT REMOVE!
import os

import numpy as np
import matplotlib.pyplot as plt

import torch

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
from pytorch_model_summary import summary

In [26]:
# Check if GPU is available and determine the device
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(f'The available device is {device}')

The available device is cpu


In [27]:
# mount drive: WE NEED IT FOR SAVING IMAGES!
#from google.colab import drive
#drive.mount('/content/gdrive')

In [28]:
# PLEASE CHANGE IT TO YOUR OWN GOOGLE DRIVE!
images_dir = 'home/orquestra/dtu/hw/results/vae/'

## Auxiliary functions

Let us define some useful log-distributions:

In [29]:
# DO NOT REMOVE
PI = torch.from_numpy(np.asarray(np.pi))
EPS = 1.e-5


def log_categorical(x, p, num_classes=256, reduction=None, dim=None):
    x_one_hot = F.one_hot(x.long(), num_classes=num_classes)
    log_p = x_one_hot * torch.log(torch.clamp(p, EPS, 1. - EPS))
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_bernoulli(x, p, reduction=None, dim=None):
    pp = torch.clamp(p, EPS, 1. - EPS)
    log_p = x * torch.log(pp) + (1. - x) * torch.log(1. - pp)
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_normal_diag(x, mu, log_var, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * log_var - 0.5 * torch.exp(-log_var) * (x - mu)**2.
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_standard_normal(x, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * x**2.
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p

## Implementing VAEs

The goal of this assignment is to implement four classes:
- `Encoder`: this class implements the encoder (variational posterior), $q_{\phi}(\mathbf{z}|\mathbf{x})$.
- `Decoder`: this class implements the decoded (the conditional likelihood), $p_{\theta}(\mathbf{x}|\mathbf{z})$.
- `Prior`: this class implements the marginal over latents (the prior), $p_{\lambda}(\mathbf{z})$.
- `VAE`: this class combines all components.

### Encoder
We start with `Encoder`. Please remember that we assume the Gaussian variational posterior with a diagonal covariance matrix.

In [30]:
# YOUR CODE GOES HERE
# NOTE: The class must containt the following functions:
# (i) reparameterization
# (ii) sample
# (iii) log_prob
# Moreover, forward must return the log-probability of the variational posterior for given x, i.e., log q(z|x)

class Encoder(nn.Module):
    def __init__(self, encoder_net):  # ADD APPROPRIATE ATTRIBUTES
        super(Encoder, self).__init__()
        self.encoder = encoder_net


    @staticmethod
    def reparameterization(mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        # FILL IN if you think it's necessary
        # output: mean, (log-)variance
        h_e = self.encoder(x)
        mu_e, log_var_e = torch.chunk(h_e, 2, dim=1)

        return mu_e, log_var_e


    def sample(self, x=None, mu_e=None, log_var_e=None):
        # FILL IN
        # output: z
        if (mu_e is None) and (log_var_e is None):
            mu_e, log_var_e = self.encode(x)
        else:
            if (mu_e is None) or (log_var_e is None):
                raise ValueError('mu and log-var cant be None!')
        z = self.reparameterization(mu_e, log_var_e)
        return z


    def log_prob(self, x=None, mu_e=None, log_var_e=None, z=None):
        if x is not None:
            mu_e, log_var_e = self.encode(x)
            z = self.sample(mu_e=mu_e,log_var_e=log_var_e)
        else:
            if (mu_e is None) or (log_var_e is None) or (z is None):
              raise ValueError('mu, log-var, and z cant be None')

        return log_normal_diag(z, mu_e, log_var_e)


    def forward(self, x, type='log_prob'):
        # FILL IN
        assert type in ['encode', 'log_prob'], 'type could be either encode or log-prob'
        if type == 'log-prob':
            return self.log_prob(x)
        else:
            return self.sample(x)

Please answer the following questions:

#### Question 1

Please explain the reparameterization trick and provide a mathematical formula.

ANSWER: The reparameterization trick is a solution to the issue of backcalculation not able to compute the derivative through a "random node". By taking an independent random variable with a simple distribution, we can perform simple operations on it such that we can express that transformation as a new 'random' variable.

Guassian variable = $z$ \\
mean = $\mu$ \\
varience = $\sigma^2$ \\
idd varible = $\epsilon \sim ℕ(\epsilon|0,1)$

\begin{equation}
  z = \mu + \sigma \cdot \epsilon
\end{equation}

By sampling $\epsilon$ from this transformation, we are able to sample $ℕ(z|\mu,\sigma)$.

#### Question 2

Please write down mathematically the log-probability of the encoder (variational posterior).




ANSWER:

$\log q_\phi(z|x) = -\frac{1}{2} \sum_{j=1}^{J} \left[ \log(2\pi) + \log(\sigma_j^2) + \frac{(z_j - \mu_j)^2}{\sigma_j^2} \right]
$



$\log q_\phi(z|x)$ denotes the log-probability of the latent variable $z$ given the input $x$


$\phi$ represents the parameters of the encoder


$J$ is the number of dimensions in the latent space


$\mu_j$ and $\sigma_j^2$ are the mean and variance for the $j$-th latent variable


$z_j$ is the $j$-th component of  $z$


The sum is taken over all latent variables.

### Decoder

The decoder is the conditional likelihood, i.e., $p(x|z)$. Please remember that we must decide on the form of the distribution (e.g., Bernoulli, Gaussian, Categorical).

In [31]:
# YOUR CODE GOES HERE
# NOTE: The class must containt the following functions:
# (i) sample
# (ii) log_prob
# Moreover, forward must return the log-probability of the conditional likelihood function for given z, i.e., log p(x|z)
# Additionally, please specify the distribution class you want to use for the decode (i.e. the `distribution` attribute)

num_vals=1

class Decoder(nn.Module):
    def __init__(self, decoder_net, distribution='bernoulli', num_vals=num_vals): # ADD APPROPRIATE ATTRIBUTES
        super(Decoder, self).__init__()

        self.decoder = decoder_net
        self.distribution = distribution
        self.num_vals=num_vals

    def decode(self, z):
        # FILL IN if you think it's necessary
        # output: parameters of the distribution p(x|z)
        h_d = self.decoder(z)

        if self.distribution == 'categorical':
          b = h_d.shape[0]
          d = h_d.shape[1]//self.num_vals
          h_d = h_d.view(b,d, self.num_vals)
          mu_d = torch.softmax(h_d, 2)
          return [mu_d]

        elif self.distribution == 'bernoulli':
            mu_d = torch.sigmoid(h_d)
            return [mu_d]
        else:
            raise ValueError('Either Catergorical or Bernoulli')

    def sample(self, z):
        # FILL IN
        # output: a sample from p(x|z)
        outs = self.decode(z)

        if self.distribution == 'categorical':
            mu_d = outs[0]
            b = mu_d.shape[0]
            m = mu_d.shape[1]
            mu_d = mu_d.view(mu_d.shape[0], -1, self.num_vals)
            p = mu_d.view(-1, self.num_vals)
            x_new = torch.multinomial(p, num_samples=1).view(b, m)

        elif self.distribution == 'bernoulli':
            mu_d = outs[0]
            x_new = torch.bernoulli(mu_d)

        else:
            raise ValueError('Either categorical or bernoulli')

        return x_new

    def log_prob(self, x, z):
        # FILL IN
        # output: log p(x|z)
        outs = self.decode(z)

        if self.distribution == 'categorical':
            mu_d = outs[0]
            log_p = log_categorical(x, mu_d, num_classes=self.num_vals, reduction='sum', dim=-1).sum(-1)

        elif self.distribution == 'bernoulli':
            mu_d = outs[0]
            log_p = log_bernoulli(x, mu_d, reduction='sum', dim=-1)

        else:
            raise ValueError('Either `categorical` or `bernoulli`')

        return log_p

    def forward(self, z, x=None, type='log_prob'):
        # FILL IN
        assert type in ['decoder', 'log_prob'], 'Type could be either decode or log_prob'
        if type == 'log_prob':
            return self.log_prob(x, z)
        else:
            return self.sample(z)

Please answer the following questions:

#### Question 3

Please explain your choice of distribution for image data used in this assignment. Additionally, please write it down mathematically (if you think that presenting it as the log-probability, then please do it).

ANSWER:

Mathematically, the Bernoulli distribution for a single pixel can be expressed as:

\begin{equation}
P(X=x)=p^{x}(1−p)^{1−x}
\end{equation}

where ( $x$ ) is the value of the pixel ($0$ or $1$), and ( $p$ ) is the probability of the pixel being $1$ (white).
When presenting this in terms of log-probability, which is often used in VAEs to compute the reconstruction loss during training, the expression becomes:

\begin{equation}
\log P(X=x)=x \log (p)+(1−x)\log (1−p)
\end{equation}

This log-probability is used because it simplifies the multiplication of probabilities into a sum of log-probabilities, which is numerically more stable and efficient for computation.

#### Question 4

Please explain how one can sample from the distribution chosen by you. Please be specific and formal (i.e., provide mathematical formulae).

ANSWER:

1. Determine the probability of success ( $p$ ), which is the probability that a pixel will be white ($1$).


2. Generate a random number ( $u$ ) from a uniform distribution in the interval $[0, 1]$.


3. Compare ( $u$ ) to ( $p$ ):

If ( $u \leq p$ ), then sample a 1 (white pixel).
If ( $u > p$ ), then sample a 0 (black pixel).



Mathematically, the sampling process can be represented as:


$X =\begin{cases}
1 & \text{if } u \leq p \\
0 & \text{if } u > p
\end{cases}$


where ( $X$ ) is the sampled value from the Bernoulli distribution.

### Prior

The prior is the marginal distribution over latent variables, i.e., $p(z)$. It plays a crucial role in the generative process and also in synthesizing images of better quality.

In this assignment, you are asked to implement a prior that is learnable (e.g., parameterized by neural networks). If you decide to implement the standard Gaussian prior only, then please be aware that you will not get any points.

For the learnable prior you can choose the **Mixture of Gaussians**.

In [32]:
# YOUR CODE GOES HERE
# NOTES:
# (i) The function "sample" must be implemented.
# (ii) The function "forward" must return the log-probability, i.e., log p(z)

class Prior(nn.Module):
    def __init__(self, L): # ADD APPROPRIATE ATTRIBUTES
        # FILL IN
        super(Prior, self).__init__()
        self.L = L

    def sample(self, batch_size=16):
        # FILL IN
        # output: a sample from p(z)
        z = torch.randn((batch_size, self.L))
        return z

    def log_prob(self, z): #I dont think this is right, check this out later
        # FILL IN
        # output: log p(z)
        return log_standard_normal(z)

#### Question 5

**Option 1:  Standard Gaussian**

- Please explain the choice of your prior and write it down mathematically.

**Option 2: Mixture of Gaussians**

Please do the following:
- Please explain the choice of your prior and write it down mathematically.
- Please write down its sampling procedure (if necessary, please add a code snippet).
- Please write down its log-probability (a mathematical formula).

1. Mixture of Gaussians (MoG) is more flexible and can model complexities in the latent space better than a standard Gaussian.

\begin{equation}p(z)= \sum_{k=1}^{K} \pi_{k}  ℕ (z|\mu_{k},\sigma_{k}^2)\end{equation}



*   $p(z)$ is the probability density of the latent variable $z$

* $\pi_k$ are the mixing coefficients
that sum to $1$

* $ℕ(z | \mu_k, \sigma_k^2)$  is the Gaussian distribution with mean $\mu_k$  and variance $\sigma_k^2$

* $K$ is the number of mixture components

2. First, select mixture compent according to the mixing coefficients, $\pi_k$, and then sampling from the selected Gaussian distribution.



```
def mog(n_samples, mixingco, means, variences):

  #choosing components
  comp_indx = np.random.choice(len(mixingco), size=n_samples, p=mixingco)

  #sample from gaussian
  samples = np.random.normal(loc=means[comp_indx], scale=np.sqrt(variences[comp_indx]))

  return samples
```


3. \begin{equation}
\log p(z) = \log \left( \sum_{k=1}^{K} exp(\log (\pi_k) + \log ℕ(z|\mu_k, \sigma_{k}^2))\right)
\end{equation}

### Complete VAE

The last class is `VAE` tha combines all components. Please remember that this class must implement the **Negative ELBO** in `forward`, as well as `sample` (*hint*: it is a composition of `sample` functions from the prior and the decoder).

In [33]:
# YOUR CODE GOES HERE
# This class combines Encoder, Decoder and Prior.
# NOTES:
# (i) The function "sample" must be implemented.
# (ii) The function "forward" must return the negative ELBO. Please remember to add an argument "reduction", which is either "mean" or "sum".
class VAE(nn.Module):
    def __init__(self, encoder_net, decoder_net, num_vals=1, L=32, likelihood_type='bernoulli'):
        super(VAE, self).__init__()

        self.encoder = Encoder(encoder_net=encoder_net)
        self.decoder = Decoder(distribution=likelihood_type, decoder_net=decoder_net, num_vals=num_vals)
        self.prior = Prior(L=L)

        self.num_vals = num_vals

        self.likelihood_type = likelihood_type

    def sample(self, batch_size=64):
        z = self.prior.sample(batch_size=batch_size)
        return self.decoder.sample(z)

    def forward(self, x, reduction='avg'):
        # FILL IN
        # output: the negative ELBO (NELBO) that is either averaged or summed (VERY IMPORTANT!)
        # encoder
        mu_e, log_var_e = self.encoder.encode(x)
        z = self.encoder.sample(mu_e=mu_e, log_var_e=log_var_e)

        # ELBO
        RE = self.decoder.log_prob(x, z)
        KL = (self.prior.log_prob(z) - self.encoder.log_prob(mu_e=mu_e, log_var_e=log_var_e, z=z)).sum(-1)

        if reduction == 'sum':
            return -(RE + KL).sum()
        else:
            return -(RE + KL).mean()


#### Question 6

Please explain your choice of the distribution for the conditional likelihood function, and write down mathematically the log-probability of the decoder.

ANSWER:

A Normal distribution would be picked if we were using continuous data. But since we are using binary data, we will continue to use a Bernoulli distribution.


\begin{equation}
\log P(x|z) = \sum_{i=1}^{N} x_i \log p_i + (1-x_i)\log(1-p_i)
\end{equation}

#### Question 7

Please write down mathematically the **Negative ELBO**.

ANSWER:
\begin{equation}
NELBO(x) = -𝔼_{q(z|x)}[\log p(x|z)]+ KL[q(z|x)||p(z)]
\end{equation}

### Evaluation and training functions

**Please DO NOT remove or modify them.**

In [34]:
# ==========DO NOT REMOVE==========

def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, (test_batch, _) in enumerate(test_loader):
        test_batch = test_batch.to(device)
        test_batch = test_batch.view(test_batch.size(0), -1)  # Flatten the batch to (batch_size, 784)
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'FINAL LOSS: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader, shape=(28,28)):
    # real images-------
    num_x = 4
    num_y = 4
    x, _ = next(iter(test_loader))
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, shape=(28,28), extra_name=''):
    x, _ = next(iter(data_loader))
    x = x.to('cpu').detach().numpy()

    # generations-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [35]:
# ==========DO NOT REMOVE==========

def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader, shape=(28,28)):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, (batch, _) in enumerate(training_loader):
            batch = batch.to(device)
            batch = batch.view(batch.size(0), -1)  # Flatten the batch to (batch_size, 784)
            loss = model.forward(batch, reduction='mean')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # save for plotting

        if e == 0:
            print('saved!')
            torch.save(model, name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model, name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, shape=shape, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### Setup

**NOTE: *Please comment your code! Especially if you introduce any new variables (e.g., hyperparameters).***

In the following cells, we define `transforms` for the dataset. Next, we initialize the data, a directory for results and some fixed hyperparameters.

In [36]:
# PLEASE DEFINE APPROPRIATE TRANFORMS FOR THE DATASET
# (If you don't see any need to do that, then you can skip this cell)
# HINT: Please prepare your data accordingly to your chosen distribution in the decoder
transforms_train = torchvision.transforms.Compose([
    transforms.ToTensor()])  # Convert the image to a PyTorch tensor
 

transforms_test = torchvision.transforms.Compose([
   transforms.ToTensor()])  # Convert the image to a PyTorch tensor

Please do not modify the code in the next cell.

In [37]:
# ==========DO NOT REMOVE==========
#-dataset
dataset = MNIST('/home/orquestra/data/MNIST/raw', train=True, download=True,
                      transform=transforms_train
                )

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [50000, 10000], generator=torch.Generator().manual_seed(14))

test_dataset = MNIST('/home/orquestra/data/MNIST/raw/', train=False, download=True,
                      transform=transforms_test
                     )
#-dataloaders
batch_size_dl = 28

train_loader = DataLoader(train_dataset, batch_size=batch_size_dl, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_dl, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size_dl, shuffle=False)

#-creating a dir for saving results
name = 'vae'
result_dir = '/home/orquestra/dtu/hw/results/vae/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)

#-hyperparams (please do not modify them!)
num_epochs = 1000 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

In the next cell, please initialize the model. Please remember about commenting your code!

In [38]:
D = 784   # input dimension
L = 28  # number of latents
M = 28 # the number of neurons in scale (s) and translation (t) nets

likelihood_type = 'bernoulli'
num_vals = 1

encoder = nn.Sequential(nn.Linear(D, M), nn.LeakyReLU(),
                        nn.Linear(M, M), nn.LeakyReLU(),
                        nn.Linear(M, L*2))

decoder = nn.Sequential(nn.Linear(L, M), nn.LeakyReLU(),
                        nn.Linear(M, M), nn.LeakyReLU(),
                        nn.Linear(M, D))


prior = Prior(L=L)#torch.distributions.MultivariateNormal(torch.zeros(L), torch.eye(L))


# Print the summary (like in Keras)
print("ENCODER:\n", summary(encoder, torch.zeros(1, D), show_input=True, show_hierarchical=True))
print("\nDECODER:\n", summary(decoder, torch.zeros(1, L), show_input=True, show_hierarchical=True))


print(f"Encoder weight shapes: {[layer.weight.shape for layer in encoder if isinstance(layer, nn.Linear)]}") #printing shapes for debugging
print(f"Decoder weight shapes: {[layer.weight.shape for layer in decoder if isinstance(layer, nn.Linear)]}") # ""
# FILL IN ANY OTHER HYPERPARAMS YOU WANT TO USE



ENCODER:
 -----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Linear-1            [1, 784]          21,980          21,980
       LeakyReLU-2             [1, 28]               0               0
          Linear-3             [1, 28]             812             812
       LeakyReLU-4             [1, 28]               0               0
          Linear-5             [1, 28]           1,624           1,624
Total params: 24,416
Trainable params: 24,416
Non-trainable params: 0
-----------------------------------------------------------------------


========================== Hierarchical Summary ==========================

Sequential(
  (0): Linear(in_features=784, out_features=28, bias=True), 21,980 params
  (1): LeakyReLU(negative_slope=0.01), 0 params
  (2): Linear(in_features=28, out_features=28, bias=True), 812 params
  (3): LeakyReLU(negative_slope=0.01), 0 params
  (4): Linear(in_fea

In [39]:
# INIT YOUR VAE (PLEASE CALL IT model)
# FILL IN
model = VAE(encoder_net=encoder, decoder_net=decoder, num_vals=1, L=L, likelihood_type=likelihood_type)
model.to(device)



VAE(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Linear(in_features=784, out_features=28, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=28, out_features=28, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=28, out_features=56, bias=True)
    )
  )
  (decoder): Decoder(
    (decoder): Sequential(
      (0): Linear(in_features=28, out_features=28, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=28, out_features=28, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=28, out_features=784, bias=True)
    )
  )
  (prior): Prior()
)

Please initialize the optimizer

In [50]:
# PLEASE DEFINE YOUR OPTIMIZER
lr = 3e-4 # learning rate (PLEASE CHANGE IT AS YOU WISH!)
betas = (0.9, 0.999)
eps = 1e-8
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr, betas=betas, eps=eps)

#### Question 8

Please explain the choice of the optimizer, and comment on the choice of the hyperparameters (e.g., the learing reate value).

ANSWER:

ADAM - common choice for VAE as it is an adaptive learning rate optimizer. It combines Adaptive Gradient Algorithm and Root Mean Square Propogation, ie; it  combines momentum and scaling of the gradient.

I added beta 1 and beta 2, as well as epsilon.

Betas control exponential decay rates. Epsilon prevents division by zero.

### Training and final evaluation

In the following two cells, we run the training and the final evaluation.

In [51]:
# ==========DO NOT REMOVE OR MODIFY==========
# Training procedure
nll_val = training(name='vae', max_patience=20,
                   num_epochs=num_epochs, model=model, optimizer=optimizer,
                   training_loader=train_loader, val_loader=val_loader,
                   shape=(28,28))

Epoch: 0, val nll=120.29863117370606
saved!
Epoch: 1, val nll=120.28796388244629
saved!
Epoch: 2, val nll=120.24742332763672
saved!
Epoch: 3, val nll=120.31174450683594
Epoch: 4, val nll=120.27745465087891
Epoch: 5, val nll=120.2750616973877
Epoch: 6, val nll=120.30310242004394
Epoch: 7, val nll=120.27761879272461
Epoch: 8, val nll=120.24426590576172
saved!
Epoch: 9, val nll=120.29404556274415
Epoch: 10, val nll=120.26303800048828
Epoch: 11, val nll=120.23999944458008
saved!
Epoch: 12, val nll=120.22974952392578
saved!
Epoch: 13, val nll=120.2666642211914
Epoch: 14, val nll=120.20893081970215
saved!
Epoch: 15, val nll=120.2164168182373
Epoch: 16, val nll=120.31710451049804
Epoch: 17, val nll=120.25669100952149
Epoch: 18, val nll=120.24151962280274
Epoch: 19, val nll=120.2675341369629
Epoch: 20, val nll=120.25488601989746
Epoch: 21, val nll=120.28549823608398
Epoch: 22, val nll=120.2364026184082
Epoch: 23, val nll=120.224358984375
Epoch: 24, val nll=120.2525501953125
Epoch: 25, val nll=

In [52]:
# ==========DO NOT REMOVE OR MODIFY==========
# Final evaluation
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)
samples_generated(result_dir + name, test_loader, extra_name='_FINAL')

plot_curve(result_dir + name, nll_val)

FINAL LOSS: nll=119.74667811279296


### Results and discussion

After a successful training of your model, we would like to ask you to present your data and analyze it. Please answer the following questions.

#### Question 9

Please select the real data, and the final generated data and include them in this report. Please comment on the following:
- Do you think the model was trained properly by looking at the generations? Please motivate your answer well.
- What are the potential problems with evaluating a generative model by looking at generated data? How can we evaluate generative models (NOTE: ELBO or NLL do not count as answers)?

ANSWER: 
1.  Subjectivity and bias will influence they way synthetic data is evaluated by a human. Also, just visually inspecting data will miss aspects of a distribution.
Looking at the generated samples from differeing points in the training and the final, you will see holes in the images.
But, in a word, no, I do not.

2. FID- Frechet Inception Score claculates distance between distributions of real and generated data and gives a quantitative assesment of the quality of generated data.


3. IS-Inception Scaore calculates the KL divergence between conditional and marginal label distributions. Measure of both the quality and diversity of new data.


4. Precision/Recall- Measure of quality by verifying 'realness'. Recall looks at the coverage of real data distribution by the generated samples.


5. KDE- Kernel Density Estimation guesses at the probability density function of the real and synthetic data, with metrics like Max Mean Discrepency MMD. It give a clear view of the closeness of the generated and real data.


6. PPL-Perceptual Path Length looks at the smoothness/consitency of the generators latent space by computing the mean perceptual distence between interpolated samples.


In [44]:
from IPython.display import IFrame
IFrame(src='vae_real_images.pdf', width=800, height=600)

In [48]:
IFrame(src='vae_generated_images_epoch_1', width=800, height=600)

In [49]:
IFrame(src='vae_generated_images_epoch_102', width=800, height=600)

In [45]:
IFrame(src='vae_generated_images_FINAL.pdf', width=800, height=600)

#### Question 10

Please include the plot of the negative ELBO. Please comment on the following:
- Is the training of your VAE stable or unstable? Why?
- What is the influence of the optimizer on your model? Do the hyperparameter values of the optimizer important and how do they influence the training? Motivate well your answer (e.g., run the script with more than one learning rate and present two plots here).

ANSWER:

No. Seeing the holes in the images throughout the training generations and the final output shows that the training is unstable.

The hyperparameters of the optimizer allow more control over how your model can be trained. By altering the learning rate you change how well the net work can learn the distripution. As seen here, by making the learning rate too small, the model will not learn. 

In [47]:
# NELBO PLot @ lr = 2e-3
IFrame(src='vae_nll_val_curve.pdf', width=800, height=600)

In [53]:
#NELBO Plot @ lr = 3e-4
IFrame(src='vae_nll_val_curve_new_lr.pdf', width=800, height=600)